In [ ]:
""" Predict-housing-prices.py

    This Python script predicts the housing prices based on some parameters

    Author: G00Z-G00Z
    Contact: edygomezgg4@gmail.com
    First created: Thu 22 Sep, 2022
"""







